In [21]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Input, Flatten
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [22]:
# 模拟数据 (CIFAR-10 数据集也可以用于此任务)
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

In [23]:
# 只选择两个类别用于二分类（如类别 0 和类别 1）
selected_classes = [0, 1]
mask_train = np.isin(y_train, selected_classes).flatten()
mask_test = np.isin(y_test, selected_classes).flatten()

In [24]:
x_train, y_train = x_train[mask_train], y_train[mask_train]
x_test, y_test = x_test[mask_test], y_test[mask_test]

In [25]:
# 正则化像素值
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

In [26]:
# 转换标签为二分类
y_train = (y_train == selected_classes[1]).astype("int")
y_test = (y_test == selected_classes[1]).astype("int")

In [27]:
# 数据增强
def data_augmentation(image):
    image = tf.image.random_flip_left_right(image)
    #image = tf.image.random_crop(image, size=[28, 28, 3])
    image = tf.image.random_crop(image, size=[32, 32, 3])    
    image = tf.image.random_brightness(image, max_delta=0.1)
    return image

def generate_augmentations(x):
    aug1 = tf.map_fn(data_augmentation, x)
    aug2 = tf.map_fn(data_augmentation, x)
    return aug1, aug2

In [28]:
# 对比学习模型 定义编码器网络和摄影头
def create_encoder():
    #base_model = tf.keras.applications.ResNet50(include_top=False, weights=None, input_shape=(28, 28, 3))
    base_model = tf.keras.applications.ResNet50(include_top=False, weights=None, input_shape=(32, 32, 3))
    x = GlobalAveragePooling2D()(base_model.output)
    encoder = Model(base_model.input, x, name="encoder")
    return encoder

def create_projection_head(encoder):
    #inputs = Input(shape=(28, 28, 3))
    inputs = Input(shape=(32, 32, 3))
    features = encoder(inputs)
    outputs = Dense(128, activation='relu')(features)
    outputs = Dense(128, activation=None)(outputs)  # 不需要激活函数
    model = Model(inputs, outputs, name="projection_head")
    return model

In [29]:
# 定义对比损失函数 实现对比学习的损失 (NT-Xent)
def contrastive_loss(projections_1, projections_2, temperature=0.5):
    # 归一化
    z1 = tf.math.l2_normalize(projections_1, axis=1)
    z2 = tf.math.l2_normalize(projections_2, axis=1)

    # 相似性计算
    similarities = tf.matmul(z1, z2, transpose_b=True)

    # 温度缩放
    logits = similarities / temperature

    # 构造标签
    batch_size = tf.shape(logits)[0]
    labels = tf.range(batch_size)

    # 计算交叉熵损失
    loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels, logits=logits)
    return tf.reduce_mean(loss)

In [30]:
# 训练对比学习模型
# 编码器和投影头
encoder = create_encoder()
projection_head = create_projection_head(encoder)

optimizer = tf.keras.optimizers.Adam()

In [31]:
@tf.function
def train_step(x_batch):
    with tf.GradientTape() as tape:
        # 生成增强数据
        aug1, aug2 = generate_augmentations(x_batch)

        # 获取投影向量
        projections_1 = projection_head(aug1, training=True)
        projections_2 = projection_head(aug2, training=True)

        # 计算对比损失
        loss = contrastive_loss(projections_1, projections_2)

    gradients = tape.gradient(loss, projection_head.trainable_variables)
    optimizer.apply_gradients(zip(gradients, projection_head.trainable_variables))
    return loss

In [32]:
# 模拟训练
batch_size = 32
epochs = 10

for epoch in range(epochs):
    for i in range(0, len(x_train), batch_size):
        x_batch = x_train[i:i+batch_size]
        loss = train_step(x_batch)
    print(f"Epoch {epoch + 1}, Loss: {loss.numpy():.4f}")

Epoch 1, Loss: 1.2228
Epoch 2, Loss: 1.1143
Epoch 3, Loss: 1.0822
Epoch 4, Loss: 1.0802
Epoch 5, Loss: 1.0527
Epoch 6, Loss: 1.0800
Epoch 7, Loss: 1.0670
Epoch 8, Loss: 1.0809
Epoch 9, Loss: 1.0542
Epoch 10, Loss: 1.0500


In [33]:
# 线性分类头微调

# 冻结编码器
encoder.trainable = False

# 构建线性分类器
inputs = Input(shape=(28, 28, 3))
features = encoder(inputs)
outputs = Dense(1, activation="sigmoid")(features)
classifier = Model(inputs, outputs, name="linear_classifier")

# 编译分类器
classifier.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

# 微调分类器
classifier.fit(x_train, y_train, validation_data=(x_test, y_test), batch_size=32, epochs=5)

# 测试分类性能
y_pred = (classifier.predict(x_test) > 0.5).astype("int")
print(f"Test Accuracy: {accuracy_score(y_test, y_pred):.2f}")

ValueError: Input 0 of layer "encoder" is incompatible with the layer: expected shape=(None, 32, 32, 3), found shape=(None, 28, 28, 3)